In [ ]:
import copy
import itertools as it
import random
import typing

from nbmetalog import nbmetalog as nbm
import opytional as opyt
import pandas as pd
import seaborn as sns
from tqdm import tqdm

import pylib

random.seed(1)

pylib.jupyter_hide_toggle()


In [ ]:
nbm.print_metadata()

pylib.jupyter_hide_toggle()


In [ ]:
def get_num_sites_reserved_per_incidence_at_rank(rank: int) -> int:
    return pylib.bit_ceil(pylib.hanoi.get_max_hanoi_value_through_index(rank) + 1)


In [ ]:
# test get_num_sites_reserved_per_incidence_at_rank
assert [get_num_sites_reserved_per_incidence_at_rank(rank) for rank in range(17)] == [
    # hanoi sequence (1-based):
    1,  # 1,
    2,  # 2,
    2,  # 1,
    4,  # 3,
    4,  # 1,
    4,  # 2,
    4,  # 1,
    4,  # 4,
    4,  # 1,
    4,  # 2,
    4,  # 1,
    4,  # 3,
    4,  # 1,
    4,  # 2,
    4,  # 1,
    8,  # 5,
    8,  # 1,
]
pylib.jupyter_hide_toggle()


In [ ]:
def get_num_incidence_reservations_at_rank(rank: int, surface_size: int) -> int:
    reservation_size = get_num_sites_reserved_per_incidence_at_rank(rank)
    assert surface_size % reservation_size == 0
    num_reservations = surface_size // reservation_size
    return num_reservations


In [ ]:
# test get_num_incidence_reservations_at_rank
assert [get_num_incidence_reservations_at_rank(rank, 64) for rank in range(17)] == [
    # hanoi sequence (1-based):
    64,  # 1,
    32,  # 2,
    32,  # 1,
    16,  # 3,
    16,  # 1,
    16,  # 2,
    16,  # 1,
    16,  # 4,
    16,  # 1,
    16,  # 2,
    16,  # 1,
    16,  # 3,
    16,  # 1,
    16,  # 2,
    16,  # 1,
    8,   # 5,
    8,   # 1,
]

pylib.jupyter_hide_toggle()


In [ ]:
def is_2x_reservation_eligible(hanoi_value: int, surface_size: int, rank: int) -> bool:
    reservation_width = get_num_sites_reserved_per_incidence_at_rank(rank)
    lb_inclusive = (
        pylib.hanoi.get_max_hanoi_value_through_index(rank) - reservation_width // 2 + 1
    )
    ub_exclusive = reservation_width // 2
    return lb_inclusive <= hanoi_value < ub_exclusive

pylib.jupyter_hide_toggle()


In [ ]:
# test is_2x_reservation_eligible
# TODO

pylib.jupyter_hide_toggle()


In [ ]:
def get_prev_invading_hanoi_value(hanoi_value: int, rank: int) -> typing.Optional[int]:
    reservation_width = get_num_sites_reserved_per_incidence_at_rank(rank)
    degrading_reservation_width = reservation_width // 2

    if hanoi_value < degrading_reservation_width // 2:
        return hanoi_value + degrading_reservation_width // 2
    else:
        return None


In [ ]:
assert get_prev_invading_hanoi_value(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
) is None

assert get_prev_invading_hanoi_value(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
) is None

assert get_prev_invading_hanoi_value(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
) is None

assert get_prev_invading_hanoi_value(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
) == 3

pylib.jupyter_hide_toggle()


In [ ]:
def is_hanoi_invader(hanoi_value: int, rank: int) -> bool:
    reservation_width = get_num_sites_reserved_per_incidence_at_rank(rank)
    return hanoi_value >= reservation_width // 2


In [ ]:
assert not is_hanoi_invader(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert is_hanoi_invader(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)

assert not is_hanoi_invader(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)

assert not is_hanoi_invader(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)

pylib.jupyter_hide_toggle()


In [ ]:
def is_hanoi_invaded(hanoi_value: int, rank: int) -> bool:
    reservation_width = get_num_sites_reserved_per_incidence_at_rank(rank)
    max_hanoi_value = pylib.hanoi.get_max_hanoi_value_through_index(rank)

    return (
        not is_hanoi_invader(hanoi_value, rank)
        and max_hanoi_value >= hanoi_value + reservation_width // 2
    )


In [ ]:
assert not is_hanoi_invaded(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert is_hanoi_invaded(
    0,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert not is_hanoi_invaded(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)

assert not is_hanoi_invaded(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)
assert is_hanoi_invaded(
    0,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)
assert not is_hanoi_invaded(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)
assert is_hanoi_invaded(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(5, 0)
)

pylib.jupyter_hide_toggle()


In [ ]:
def is_hanoi_invadable_and_uninvaded(hanoi_value: int, rank: int) -> bool:
    return (
        not is_hanoi_invader(hanoi_value, rank)
        and not is_hanoi_invaded(hanoi_value, rank)
    )


In [ ]:
assert is_hanoi_invadable_and_uninvaded(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert not is_hanoi_invadable_and_uninvaded(
    2,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert not is_hanoi_invadable_and_uninvaded(
    0,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert not is_hanoi_invadable_and_uninvaded(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert is_hanoi_invadable_and_uninvaded(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)
assert not is_hanoi_invadable_and_uninvaded(
    0,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)
assert is_hanoi_invadable_and_uninvaded(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)
assert not is_hanoi_invadable_and_uninvaded(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(5, 0)
)
assert not is_hanoi_invadable_and_uninvaded(
    5,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(5, 0)
)
pylib.jupyter_hide_toggle()


In [ ]:
def get_next_invading_hanoi_value(hanoi_value: int, rank: int) -> int:
    reservation_width = get_num_sites_reserved_per_incidence_at_rank(rank)
    degrading_reservation_width = reservation_width // 2

    if hanoi_value < degrading_reservation_width:
        return hanoi_value + degrading_reservation_width * 2
    else:
        return hanoi_value + reservation_width * 2


In [ ]:
assert get_next_invading_hanoi_value(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
) == 5

assert get_next_invading_hanoi_value(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
) == 11

assert get_next_invading_hanoi_value(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
) == 11

assert get_next_invading_hanoi_value(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
) == 9

assert get_next_invading_hanoi_value(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(5, 0)
) == 9

pylib.jupyter_hide_toggle()


In [ ]:
def get_invading_hanoi_value(hanoi_value: int, rank: int) -> int:
    reservation_width = get_num_sites_reserved_per_incidence_at_rank(rank)
    degrading_reservation_width = reservation_width // 2

    if hanoi_value < degrading_reservation_width:
        return hanoi_value + degrading_reservation_width
    else:
        return hanoi_value + reservation_width


In [ ]:
assert get_invading_hanoi_value(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
) == 3

assert get_invading_hanoi_value(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
) == 7

assert get_invading_hanoi_value(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
) == 7

pylib.jupyter_hide_toggle()


In [ ]:
def get_rank_of_hanoi_invasion(hanoi_value: int, rank: int) -> int:
    invading_hanoi_value = get_invading_hanoi_value(hanoi_value, rank)
    return pylib.hanoi.get_index_of_hanoi_value_nth_incidence(
        invading_hanoi_value,
        0,
    )


In [ ]:
assert get_rank_of_hanoi_invasion(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
) == pylib.hanoi.get_index_of_hanoi_value_nth_incidence(3, 0)

assert get_rank_of_hanoi_invasion(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
) == pylib.hanoi.get_index_of_hanoi_value_nth_incidence(7, 0)

assert get_rank_of_hanoi_invasion(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
) == pylib.hanoi.get_index_of_hanoi_value_nth_incidence(7, 0)

pylib.jupyter_hide_toggle()


In [ ]:
def get_rank_of_next_hanoi_invasion(hanoi_value: int, rank: int) -> int:
    next_invading_hanoi_value = get_next_invading_hanoi_value(hanoi_value, rank)
    return pylib.hanoi.get_index_of_hanoi_value_nth_incidence(
        next_invading_hanoi_value,
        0,
    )


In [ ]:
assert get_rank_of_next_hanoi_invasion(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
) == pylib.hanoi.get_index_of_hanoi_value_nth_incidence(5, 0)

assert get_rank_of_next_hanoi_invasion(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
) == pylib.hanoi.get_index_of_hanoi_value_nth_incidence(11, 0)

assert get_rank_of_next_hanoi_invasion(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
) == pylib.hanoi.get_index_of_hanoi_value_nth_incidence(11, 0)

assert get_rank_of_next_hanoi_invasion(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
) == pylib.hanoi.get_index_of_hanoi_value_nth_incidence(9, 0)

assert get_rank_of_next_hanoi_invasion(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(5, 0)
) == pylib.hanoi.get_index_of_hanoi_value_nth_incidence(9, 0)

pylib.jupyter_hide_toggle()


In [ ]:
def get_rank_of_prev_hanoi_invasion(hanoi_value: int, rank: int) -> typing.Optional[int]:
    prev_invading_hanoi_value = get_prev_invading_hanoi_value(hanoi_value, rank)
    return opyt.apply_if(
        prev_invading_hanoi_value,
        lambda x: pylib.hanoi.get_index_of_hanoi_value_nth_incidence(x, 0),
    )


In [ ]:
assert get_rank_of_prev_hanoi_invasion(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
) is None

assert get_rank_of_prev_hanoi_invasion(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
) is None

assert get_rank_of_prev_hanoi_invasion(
    3,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
) is None

assert get_rank_of_prev_hanoi_invasion(
    1,
    pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
) == pylib.hanoi.get_index_of_hanoi_value_nth_incidence(3, 0)

pylib.jupyter_hide_toggle()


In [ ]:
def get_hanoi_incidences_until_invasion(hanoi_value: int, rank: int) -> int:
    invasion_rank = get_rank_of_hanoi_invasion(hanoi_value, rank)
    if invasion_rank > rank:
        hanoi_value_cadence = pylib.hanoi.get_hanoi_value_index_cadence(hanoi_value)
        return (invasion_rank - rank) // hanoi_value_cadence
    else:
        return 0


In [ ]:
def get_upcoming_hanoi_invasion_value(hanoi_value: int, rank: int) -> int:
    rank = max(
        rank,
        pylib.hanoi.get_index_of_hanoi_value_nth_incidence(hanoi_value, 0),
    )

    reservation_width = get_num_sites_reserved_per_incidence_at_rank(rank)
    if is_hanoi_invader(hanoi_value, rank) or is_hanoi_invaded(hanoi_value, rank):
        return hanoi_value + reservation_width
    else:
        return hanoi_value + reservation_width // 2


In [ ]:
def get_upcoming_hanoi_invasion_rank(hanoi_value: int, rank: int) -> int:
    upcoming_invasion_hanoi_value = get_upcoming_hanoi_invasion_value(hanoi_value, rank)
    upcoming_invasion_rank = pylib.hanoi.get_index_of_hanoi_value_nth_incidence(
        upcoming_invasion_hanoi_value,
        0
    )
    return upcoming_invasion_rank


In [ ]:
def get_regime_end_rank(hanoi_value: int, rank: int) -> int:
    return get_upcoming_hanoi_invasion_rank(hanoi_value, rank)


In [ ]:
def get_regime_mx(hanoi_value: int, rank: int) -> int:
    if is_hanoi_invadable_and_uninvaded(hanoi_value, rank):
        return 2
    else:
        return 1


In [ ]:
def get_regime_num_reservations_available(hanoi_value: int, surface: int, rank: int) -> int:
    mx = get_regime_mx(hanoi_value, rank)
    return get_num_incidence_reservations_at_rank(rank, surface_size) * mx


In [ ]:
def get_regime_reservation_downgrade_rank(hanoi_value: int, surface_size: int, rank: int) -> int:
    cadence = pylib.hanoi.get_hanoi_value_index_cadence(hanoi_value)
    offset = pylib.hanoi.get_hanoi_value_index_offset(hanoi_value)
    big_tour_size = get_regime_num_reservations_available(
        hanoi_value,
        surface_size,
        rank,
    )
    big_tour_time = big_tour_size * cadence
    end_rank = get_regime_end_rank(hanoi_value, rank)
    assert end_rank

    #    16 big tour
    #    8 little tour

    #    64 end rank -> 56 downgrade rank
    #    65 end rank -> 56 downgrade rank
    #    72 end_rank -> 72 downgrade rank
    required_lag = big_tour_time // 2
    # oc is offset corrected
    end_rank_oc = end_rank - offset
    deadline_rank_oc = end_rank_oc - required_lag
    #
    #   call this tt_split |        | call this tt_above
    #########################################################
    #             |----tour time----|
    #             |        :        |
    #                           ^deadline rank
    #                      ^mapped to here
    #                                     ^ end_rank
    #########################################################
    #             |----tour time----|
    #             |        :        |        :        |
    #                                   ^deadline rank
    #                      ^mapped to here
    #                                              ^ end_rank
    tt_above_oc = (
        deadline_rank_oc + big_tour_time // 2 - 1
    ) - (
        (deadline_rank_oc + big_tour_time // 2) % big_tour_time
    )
    tt_split_oc = tt_above_oc - big_tour_time // 2
    assert deadline_rank_oc - tt_split_oc <= big_tour_time

    downgrade_rank_oc = tt_split_oc
    downgrade_rank = downgrade_rank_oc + offset
    assert downgrade_rank <= end_rank
    assert big_tour_time // 2 <= end_rank - downgrade_rank <= big_tour_time + big_tour_time // 2
    assert (
        (pylib.hanoi.get_incidence_count_of_hanoi_value_through_index(hanoi_value, downgrade_rank))
        % big_tour_size
        == big_tour_size // 2
    )

    return downgrade_rank


In [ ]:
def get_regime_num_reservations_provided(hanoi_value: int, surface_size: int, rank: int) -> int:
    thresh = get_regime_reservation_downgrade_rank(hanoi_value, surface_size, rank)
    before_thresh_num = get_regime_num_reservations_available(
        hanoi_value,
        surface_size,
        rank,
    )
    if rank >= thresh:
        return before_thresh_num // 2
    else:
        return before_thresh_num


In [ ]:
def get_deposition_site_at_rank(rank: int, surface_size: int) -> int:
    within_reservation_offset = pylib.hanoi.get_hanoi_value_at_index(rank)

    num_incidence_reservations = get_regime_num_reservations_provided(
        within_reservation_offset, surface_size, rank
    )

    reservation_index = (
        pylib.hanoi.get_hanoi_value_incidence_at_index(rank)
        % num_incidence_reservations
    )

    longevity_ordered_reservation_position = (
        # this is more correct in a theoretical sense, and does have some effect
        # but actual functional improvements relative to simpler naive/alternating
        # should be tested; it may not be worth the extra complexity
        pylib.longevity_ordering_piecewise_ascending.get_longevity_mapped_position_of_index(
            reservation_index,
            surface_size,
#             polarity=True,
        )
    )

    res = (
        longevity_ordered_reservation_position
        + within_reservation_offset
    )
    assert res < surface_size
    return res


In [ ]:
# test visualization

surface_size = 16
num_generations = 500

surface_timestamps = [0] * surface_size
surface_values = [random.choice([0, 1]) for __ in range(surface_size)]

surface_timestamps_records = []
for generation in tqdm(range(num_generations)):
    target_site = get_deposition_site_at_rank(generation, surface_size)
    surface_values[target_site] = random.choice([0, 1])
    surface_timestamps[target_site] = generation

#     assert [
#         get_deposition_rank_at_site(site, surface_size, generation + 1)
#         for site in range(surface_size)
#     ] == surface_timestamps

    for site, timestamp in enumerate(surface_timestamps):
        surface_timestamps_records.append(
            {
                "generation": generation,
                "site": site,
                "timestamp": timestamp,
            }
        )

surface_timestamps_df = pd.DataFrame.from_records(surface_timestamps_records)
surface_timestamps_df

sns.scatterplot(
    data=surface_timestamps_df,
    x="timestamp",
    y="generation",
    hue="site",
).invert_yaxis()


In [ ]:
pylib.jupyter_hide_toggle()

# test get_deposition_site_at_rank
# **needs to be updated**
# assert [get_deposition_site_at_rank(rank, 64) for rank in range(23)] == [
#          # num_reservations
#          #     # hanoi sequence (0-based):
#     0,   # 64, # 0,
#     1,   # 32, # 1,
#     32,  # 32, # 0,
#     2,   # 16, # 2,
#     16,  # 16, # 0,
#     33,  # 16, # 1,
#     48,  # 16, # 0,
#     3,   # 16, # 3,
#     8,   # 16, # 0,
#     17,  # 16, # 1,
#     24,  # 16, # 0,
#     34,  # 16, # 2,
#     40,  # 16, # 0,
#     49,  # 16, # 1,
#     56,  # 16, # 0,
#     4,   # 8,  # 4,
#     0,   # 8,  # 0,
#     9,   # 8,  # 1,
#     32,  # 8,  # 0,
#     18,  # 8,  # 2,
#     16,  # 8,  # 0,
#     25,  # 8,  # 1,
#     48,  # 8,  # 0,
# ]

# assert [get_deposition_site_at_rank(rank, 16) for rank in range(21)] == [
#         # num_reservations
#         #     # hanoi sequence (0-based):
#     0,  # 16, # 0,
#     1,  # 8,  # 1,
#     8,  # 8,  # 0,
#     2,  # 4,  # 2,
#     4,  # 4,  # 0,
#     9,  # 4,  # 1,
#     12, # 4,  # 0,
#     3,  # 4,  # 3,
#     0,  # 4,  # 0,
#     5,  # 4,  # 1,
#     8,  # 4,  # 0,
#     10, # 4,  # 2,
#     4,  # 4,  # 0,
#     13, # 4,  # 1,
#     12, # 4,  # 0,
#     4,  # 2,  # 4,
#     0,  # 2,  # 0,
#     1,  # 2,  # 1,
#     8,  # 2,  # 0,
#     6,  # 2,  # 2,
#     0,  # 2,  # 0,
# ]


In [ ]:
def is_site_reservation_holdover_at_rank(site: int, surface_size: int, rank: int) -> bool:
    reservation_size = get_num_sites_reserved_per_incidence_at_rank(rank)
    within_reservation_offset = site % reservation_size

    return (
        within_reservation_offset
        > pylib.hanoi.get_max_hanoi_value_through_index(rank)
    )


In [ ]:
# test get_hanoi_value_of_site

# rank
#    # deposition site
#    #     # num_reservations
#    #     #     # hanoi sequence (0-based):
# 0  # 0,  # 16, # 0,
for rank in range(3):
    assert [
        is_site_reservation_holdover_at_rank(site, 16, rank) for site in range(16)
    ] == [False] * 16

# rank
#    # deposition site
#    #     # num_reservations
#    #     #     # hanoi sequence (0-based):
# 3  # 2,  # 4,  # 2,
# 4  # 4,  # 4,  # 0,
# 5  # 9,  # 4,  # 1,
# 6  # 12, # 4,  # 0,
for rank in range(3, 7):
    assert [is_site_reservation_holdover_at_rank(site, 16, rank) for site in range(16)] == [
        False,
        False,
        False,
        True,
    ] * 4

# rank
#    # deposition site
#    #     # num_reservations
#    #     #     # hanoi sequence (0-based):
# 7  # 3,  # 4,  # 3,
# 8  # 0,  # 4,  # 0,
# 9  # 5,  # 4,  # 1,
# 10 # 8,  # 4,  # 0,
# 11 # 10, # 4,  # 2,
# 12 # 4,  # 4,  # 0,
# 13 # 13, # 4,  # 1,
# 14 # 12, # 4,  # 0,
for rank in range(7, 14):
    assert [
        is_site_reservation_holdover_at_rank(site, 16, rank) for site in range(16)
    ] == [False] * 16

# rank
#    # deposition site
#    #     # num_reservations
#    #     #     # hanoi sequence (0-based):
# 15 # 4,  # 2,  # 4,
# 16 # 0,  # 2,  # 0,
# 17 # 1,  # 2,  # 1,
# 18 # 8,  # 2,  # 0,
# 19 # 6,  # 2,  # 2,
# 20 # 0,  # 2,  # 0,
for rank in range(15, 21):
    assert [is_site_reservation_holdover_at_rank(site, 16, rank) for site in range(16)] == [
        False,
        False,
        False,
        False,
        False,
        True,
        True,
        True,
    ] * 2

pylib.jupyter_hide_toggle()


In [ ]:
def get_hanoi_value_of_site_at_rank(site: int, surface_size: int, rank: int) -> int:
    is_reservation_holdover = (
        is_site_reservation_holdover_at_rank(site, surface_size, rank)
    )

    reservation_size = get_num_sites_reserved_per_incidence_at_rank(rank)
    if is_reservation_holdover:
        reservation_size //= 2

    return site % reservation_size

pylib.jupyter_hide_toggle()


In [ ]:
# test get_hanoi_value_of_site

# rank
#    # deposition site
#    #     # num_reservations
#    #     #     # hanoi sequence (0-based):
# 0  # 0,  # 16, # 0,
assert [get_hanoi_value_of_site_at_rank(site, 16, 0) for site in range(16)] == [0] * 16

# rank
#    # deposition site
#    #     # num_reservations
#    #     #     # hanoi sequence (0-based):
# 1  # 1,  # 8,  # 1,
# 2  # 8,  # 8,  # 0,
for rank in 1, 2:
    assert [get_hanoi_value_of_site_at_rank(site, 16, rank) for site in range(16)] == [
        0,
        1,
    ] * 8

# rank
#    # deposition site
#    #     # num_reservations
#    #     #     # hanoi sequence (0-based):
# 3  # 2,  # 4,  # 2,
# 4  # 4,  # 4,  # 0,
# 5  # 9,  # 4,  # 1,
# 6  # 12, # 4,  # 0,
for rank in range(3, 7):
    assert [get_hanoi_value_of_site_at_rank(site, 16, rank) for site in range(16)] == [
        0,
        1,
        2,
        1,
    ] * 4

# rank
#    # deposition site
#    #     # num_reservations
#    #     #     # hanoi sequence (0-based):
# 7  # 3,  # 4,  # 3,
# 8  # 0,  # 4,  # 0,
# 9  # 5,  # 4,  # 1,
# 10 # 8,  # 4,  # 0,
# 11 # 10, # 4,  # 2,
# 12 # 4,  # 4,  # 0,
# 13 # 13, # 4,  # 1,
# 14 # 12, # 4,  # 0,
for rank in range(7, 14):
    assert [get_hanoi_value_of_site_at_rank(site, 16, rank) for site in range(16)] == [
        0,
        1,
        2,
        3,
    ] * 4

# rank
#    # deposition site
#    #     # num_reservations
#    #     #     # hanoi sequence (0-based):
# 15 # 4,  # 2,  # 4,
# 16 # 0,  # 2,  # 0,
# 17 # 1,  # 2,  # 1,
# 18 # 8,  # 2,  # 0,
# 19 # 6,  # 2,  # 2,
# 20 # 0,  # 2,  # 0,
for rank in range(15, 21):
    assert [get_hanoi_value_of_site_at_rank(site, 16, rank) for site in range(16)] == [
        0,
        1,
        2,
        3,
        4,
        1,
        2,
        3,
    ] * 2

pylib.jupyter_hide_toggle()


In [ ]:
def get_reservation_index_of_site_at_rank(
    site: int, surface_size: int, rank: int
) -> int:
    reservation_size = get_num_sites_reserved_per_incidence_at_rank(rank)
    num_incidence_reservations = get_num_incidence_reservations_at_rank(
        rank, surface_size
    )

    is_reservation_holdover = (
        is_site_reservation_holdover_at_rank(site, surface_size, rank)
    )
    if is_reservation_holdover:
        reservation_size //= 2
        num_incidence_reservations *= 2

    longevity_ordered_reservation_position = site // reservation_size

    reservation_index = pylib.longevity_ordering_piecewise_ascending.get_longevity_index_of_mapped_position(
        longevity_ordered_reservation_position,
        num_incidence_reservations,
        polarity=True,
    )
    return reservation_index

pylib.jupyter_hide_toggle()


In [ ]:
pylib.jupyter_hide_toggle()

# test get_reservation_index_of_site_at_rank
# **needs to be updated**
# assert [get_reservation_index_of_site_at_rank(site, 16, 0) for site in range(16)] == [
#     0,
#     8,
#     4,
#     9,
#     2,
#     10,
#     5,
#     11,
#     1,
#     12,
#     6,
#     13,
#     3,
#     14,
#     7,
#     15,
# ]

# for rank in 1, 2:
#     assert [get_reservation_index_of_site_at_rank(site, 16, rank) for site in range(16)] == [
#         0,
#         0,
#         4,
#         4,
#         2,
#         2,
#         5,
#         5,
#         1,
#         1,
#         6,
#         6,
#         3,
#         3,
#         7,
#         7,
#     ]

# for rank in range(3, 7):
#     assert [get_reservation_index_of_site_at_rank(site, 16, rank) for site in range(16)] == [
#         0,
#         0,
#         0,
#         4,
#         2,
#         2,
#         2,
#         5,
#         1,
#         1,
#         1,
#         6,
#         3,
#         3,
#         3,
#         7,
#     ]

# for rank in range(7, 14):
#     assert [get_reservation_index_of_site_at_rank(site, 16, rank) for site in range(16)] == [
#         0,
#         0,
#         0,
#         0,
#         2,
#         2,
#         2,
#         2,
#         1,
#         1,
#         1,
#         1,
#         3,
#         3,
#         3,
#         3,
#     ]

# for rank in range(15, 21):
#     assert [get_reservation_index_of_site_at_rank(site, 16, 15) for site in range(16)] == [
#         0,
#         0,
#         0,
#         0,
#         0,
#         2,
#         2,
#         2,
#         1,
#         1,
#         1,
#         1,
#         1,
#         3,
#         3,
#         3,
#     ]


In [ ]:
def get_hanoi_incidence_of_site_at_rank(site: int, surface_size: int, rank: int) -> int:
    # this function deals with incidence wraparound over number of reservations
    hanoi_value = get_hanoi_value_of_site_at_rank(site, surface_size, rank)
    incidence_count = pylib.hanoi.get_incidence_count_of_hanoi_value_through_index(
        hanoi_value,
        rank,
    )
    incidence_count = max(incidence_count, 1)
    max_incidence = incidence_count - 1

    num_incidence_reservations_naive = get_num_incidence_reservations_at_rank(
        rank, surface_size
    )
    is_site_reservation_holdover_at_rank_ = is_2x_reservation_eligible(
        hanoi_value,
        surface_size,
        rank,
    )
    num_incidence_reservations = (
        num_incidence_reservations_naive
        * (is_site_reservation_holdover_at_rank_ + 1)
    )

    reservation_index = get_reservation_index_of_site_at_rank(site, surface_size, rank)
    assert reservation_index < num_incidence_reservations

    if max_incidence >= num_incidence_reservations:
        excess_incidence = (
            max_incidence - reservation_index
        ) % num_incidence_reservations
        candidate_incidence = max_incidence - excess_incidence
        candidate_rank = pylib.hanoi.get_index_of_hanoi_value_nth_incidence(
            hanoi_value,
            candidate_incidence,
        )
        assert candidate_rank <= rank
        if candidate_rank == rank:
            return candidate_incidence
        else:
            return get_hanoi_incidence_of_site_at_rank(
                site, surface_size, candidate_rank
            )
    else:
        return reservation_index

pylib.jupyter_hide_toggle()


In [ ]:
pylib.jupyter_hide_toggle()


# test get_hanoi_incidence_of_site_at_rank
# **needs to be updated**
# assert [get_hanoi_incidence_of_site_at_rank(0, 64, rank) for rank in range(20)] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # 0,  # 64, # 0,
#     0,  # 1,  # 32, # 1,
#     0,  # 32, # 32, # 0,
#     0,  # 2,  # 16, # 2,
#     0,  # 16, # 16, # 0,
#     0,  # 33, # 16, # 1,
#     0,  # 48, # 16, # 0,
#     0,  # 3,  # 16, # 3,
#     0,  # 8,  # 16, # 0,
#     0,  # 17, # 16, # 1,
#     0,  # 24, # 16, # 0,
#     0,  # 34, # 16, # 2,
#     0,  # 40, # 16, # 0,
#     0,  # 49, # 16, # 1,
#     0,  # 56, # 16, # 0,
#     0,  # 4,  # 8,  # 4,
#     8,  # 0,  # 8,  # 0,
#     8,  # 9,  # 8,  # 1,
#     8,  # 32, # 8,  # 0,
#     8,  # 25, # 8,  # 1,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(1, 64, rank) for rank in range(20)] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     32, # 0,  # 64, # 0,
#     0,  # 1,  # 32, # 1,
#     0,  # 32, # 32, # 0,
#     0,  # 2,  # 16, # 2,
#     0,  # 16, # 16, # 0,
#     0,  # 33, # 16, # 1,
#     0,  # 48, # 16, # 0,
#     0,  # 3,  # 16, # 3,
#     0,  # 8,  # 16, # 0,
#     0,  # 17, # 16, # 1,
#     0,  # 24, # 16, # 0,
#     0,  # 34, # 16, # 2,
#     0,  # 40, # 16, # 0,
#     0,  # 49, # 16, # 1,
#     0,  # 56, # 16, # 0,
#     0,  # 4,  # 8,  # 4,
#     0,  # 0,  # 8,  # 0,
#     0,  # 9,  # 8,  # 1,
#     0,  # 32, # 8,  # 0,
#     0,  # 25, # 8,  # 1,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(32, 64, rank) for rank in range(20)] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     1,  # 0,  # 64, # 0,
#     1,  # 1,  # 32, # 1,
#     1,  # 32, # 32, # 0,
#     1,  # 2,  # 16, # 2,
#     1,  # 16, # 16, # 0,
#     1,  # 33, # 16, # 1,
#     1,  # 48, # 16, # 0,
#     1,  # 3,  # 16, # 3,
#     1,  # 8,  # 16, # 0,
#     1,  # 17, # 16, # 1,
#     1,  # 24, # 16, # 0,
#     1,  # 34, # 16, # 2,
#     1,  # 40, # 16, # 0,
#     1,  # 49, # 16, # 1,
#     1,  # 56, # 16, # 0,
#     1,  # 4,  # 8,  # 4,
#     1,  # 0,  # 8,  # 0,
#     1,  # 9,  # 8,  # 1,
#     9,  # 32, # 8,  # 0,
#     9,  # 25, # 8,  # 1,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(63, 64, rank) for rank in range(20)] == [
#          # deposition site
#          #     # num_reservations
#          #     #     # hanoi sequence (0-based):
#     63,  # 0,  # 64, # 0,
#     31,  # 1,  # 32, # 1,
#     31,  # 32, # 32, # 0,
#     31,  # 2,  # 16, # 2,
#     31,  # 16, # 16, # 0,
#     31,  # 33, # 16, # 1,
#     31,  # 48, # 16, # 0,
#     15,  # 3,  # 16, # 3,
#     15,  # 8,  # 16, # 0,
#     15,  # 17, # 16, # 1,
#     15,  # 24, # 16, # 0,
#     15,  # 34, # 16, # 2,
#     15,  # 40, # 16, # 0,
#     15,  # 49, # 16, # 1,
#     15,  # 56, # 16, # 0,
#     15,  # 4,  # 8,  # 4,
#     15,  # 0,  # 8,  # 0,
#     15,  # 9,  # 8,  # 1,
#     15,  # 32, # 8,  # 0,
#     15,  # 25, # 8,  # 1,
# ]


In [ ]:
pylib.jupyter_hide_toggle()

# test get_hanoi_incidence_of_site_at_rank
# **needs to be updated**
# assert [get_hanoi_incidence_of_site_at_rank(0, 16, rank) for rank in range(21)] == [
#         # deposition_site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     0,  # 8,  # 8,  # 0,
#     0,  # 2,  # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     0,  # 9,  # 4,  # 1,
#     0,  # 12, # 4,  # 0,
#     0,  # 3,  # 4,  # 3,
#     4,  # 0,  # 4,  # 0,
#     4,  # 5,  # 4,  # 1,
#     4,  # 8,  # 4,  # 0,
#     4,  # 10, # 4,  # 2,
#     4,  # 4,  # 4,  # 0,
#     4,  # 13, # 4,  # 1,
#     4,  # 12, # 4,  # 0,
#     4,  # 4,  # 2,  # 4,
#     8,  # 0,  # 2,  # 0,
#     8,  # 1,  # 2,  # 1,
#     8,  # 8,  # 2,  # 0,
#     8,  # 6,  # 2,  # 2,
#     10, # 0,  # 2,  # 0,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(1, 16, rank) for rank in range(21)] == [
#         # deposition_site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     8,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     0,  # 8,  # 8,  # 0,
#     0,  # 2,  # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     0,  # 9,  # 4,  # 1,
#     0,  # 12, # 4,  # 0,
#     0,  # 3,  # 4,  # 3,
#     0,  # 0,  # 4,  # 0,
#     0,  # 5,  # 4,  # 1,
#     0,  # 8,  # 4,  # 0,
#     0,  # 10, # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     0,  # 13, # 4,  # 1,
#     0,  # 12, # 4,  # 0,
#     0,  # 4,  # 2,  # 4,
#     0,  # 0,  # 2,  # 0,
#     4,  # 1,  # 2,  # 1,
#     4,  # 8,  # 2,  # 0,
#     4,  # 6,  # 2,  # 2,
#     4,  # 0,  # 2,  # 0,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(2, 16, rank) for rank in range(21)] == [
#         # rank
#         #    # deposition_site
#         #    #     # num_reservations
#         #    #     #     # hanoi sequence (0-based):
#     4,  # 0  # 0,  # 16, # 0,
#     4,  # 1  # 1,  # 8,  # 1,
#     4,  # 2  # 8,  # 8,  # 0,
#     0,  # 3  # 2,  # 4,  # 2,
#     0,  # 4  # 4,  # 4,  # 0,
#     0,  # 5  # 9,  # 4,  # 1,
#     0,  # 6  # 12, # 4,  # 0,
#     0,  # 7  # 3,  # 4,  # 3,
#     0,  # 8  # 0,  # 4,  # 0,
#     0,  # 9  # 5,  # 4,  # 1,
#     0,  # 10 # 8,  # 4,  # 0,
#     0,  # 11 # 10, # 4,  # 2,
#     0,  # 12 # 4,  # 4,  # 0,
#     0,  # 13 # 13, # 4,  # 1,
#     0,  # 14 # 12, # 4,  # 0,
#     0,  # 15 # 4,  # 2,  # 4,
#     0,  # 16 # 0,  # 2,  # 0,
#     0,  # 17 # 1,  # 2,  # 1,
#     0,  # 18 # 8,  # 2,  # 0,
#     0,  # 19 # 6,  # 2,  # 2,
#     0,  # 20 # 0,  # 2,  # 0,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(8, 16, rank) for rank in range(21)] == [
#         # deposition_site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     1,  # 0,  # 16, # 0,
#     1,  # 1,  # 8,  # 1,
#     1,  # 8,  # 8,  # 0,
#     1,  # 2,  # 4,  # 2,
#     1,  # 4,  # 4,  # 0,
#     1,  # 9,  # 4,  # 1,
#     1,  # 12, # 4,  # 0,
#     1,  # 3,  # 4,  # 3,
#     1,  # 0,  # 4,  # 0,
#     1,  # 5,  # 4,  # 1,
#     5,  # 8,  # 4,  # 0,
#     5,  # 10, # 4,  # 2,
#     5,  # 4,  # 4,  # 0,
#     5,  # 13, # 4,  # 1,
#     5,  # 12, # 4,  # 0,
#     5,  # 4,  # 2,  # 4,
#     5,  # 0,  # 2,  # 0,
#     5,  # 1,  # 2,  # 1,
#     9,  # 8,  # 2,  # 0,
#     9,  # 6,  # 2,  # 2,
#     9,  # 0,  # 2,  # 0,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(15, 16, rank) for rank in range(21)] == [
#         # deposition_site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     15, # 0,  # 16, # 0,
#     7,  # 1,  # 8,  # 1,
#     7,  # 8,  # 8,  # 0,
#     7,  # 2,  # 4,  # 2,
#     7,  # 4,  # 4,  # 0,
#     7,  # 9,  # 4,  # 1,
#     7,  # 12, # 4,  # 0,
#     3,  # 3,  # 4,  # 3,
#     3,  # 0,  # 4,  # 0,
#     3,  # 5,  # 4,  # 1,
#     3,  # 8,  # 4,  # 0,
#     3,  # 10, # 4,  # 2,
#     3,  # 4,  # 4,  # 0,
#     3,  # 13, # 4,  # 1,
#     3,  # 12, # 4,  # 0,
#     3,  # 4,  # 2,  # 4,
#     3,  # 0,  # 2,  # 0,
#     3,  # 1,  # 2,  # 1,
#     3,  # 8,  # 2,  # 0,
#     3,  # 6,  # 2,  # 2,
#     3,  # 0,  # 2,  # 0,
# ]


In [ ]:
# def get_deposition_rank_at_site(
#     site: int, surface_size: int, num_depositions: int
# ) -> int:
#     if num_depositions == 0:
#         return 0

#     rank = num_depositions - 1

#     hanoi_value = get_hanoi_value_of_site_at_rank(site, surface_size, rank)
#     hanoi_incidence = get_hanoi_incidence_of_site_at_rank(site, surface_size, rank)
#     candidate_rank = pylib.hanoi.get_index_of_hanoi_value_nth_incidence(
#         hanoi_value,
#         hanoi_incidence,
#     )

#     if candidate_rank < num_depositions:
#         return candidate_rank
#     else:
#         # haven't got this far in the reservation
#         # what was left here the last time around
#         # todo this recursion is slow and inefficient
#         return get_deposition_rank_at_site(site, surface_size, num_depositions - 1)

pylib.jupyter_hide_toggle()


In [ ]:
# test get_deposition_rank_at_site
# assert [
#     get_deposition_rank_at_site(0, 16, num_depositions) for num_depositions in range(22)
# ] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # ~~~~ n/a ~~~~
#     0,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     0,  # 8,  # 8,  # 0,
#     0,  # 2,  # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     0,  # 9,  # 4,  # 1,
#     0,  # 12, # 4,  # 0,
#     0,  # 3,  # 4,  # 3,
#     8,  # 0,  # 4,  # 0,
#     8,  # 5,  # 4,  # 1,
#     8,  # 8,  # 4,  # 0,
#     8,  # 10, # 4,  # 2,
#     8,  # 4,  # 4,  # 0,
#     8,  # 13, # 4,  # 1,
#     8,  # 12, # 4,  # 0,
#     8,  # 4,  # 2,  # 4,
#     16, # 0,  # 2,  # 0,
#     16, # 1,  # 2,  # 1,
#     16, # 8,  # 2,  # 0,
#     16, # 6,  # 2,  # 2,
#     20, # 0,  # 2,  # 0,
# ]
# assert [
#     get_deposition_rank_at_site(1, 16, num_depositions) for num_depositions in range(22)
# ] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # ~~~~ n/a ~~~~
#     0,  # 0,  # 16, # 0,
#     1,  # 1,  # 8,  # 1,
#     1,  # 8,  # 8,  # 0,
#     1,  # 2,  # 4,  # 2,
#     1,  # 4,  # 4,  # 0,
#     1,  # 9,  # 4,  # 1,
#     1,  # 12, # 4,  # 0,
#     1,  # 3,  # 4,  # 3,
#     1,  # 0,  # 4,  # 0,
#     1,  # 5,  # 4,  # 1,
#     1,  # 8,  # 4,  # 0,
#     1,  # 10, # 4,  # 2,
#     1,  # 4,  # 4,  # 0,
#     1,  # 13, # 4,  # 1,
#     1,  # 12, # 4,  # 0,
#     1,  # 4,  # 2,  # 4,
#     1,  # 0,  # 2,  # 0,
#     17, # 1,  # 2,  # 1,
#     17, # 8,  # 2,  # 0,
#     17, # 6,  # 2,  # 2,
#     17, # 0,  # 2,  # 0,
# ]
# assert [
#     get_deposition_rank_at_site(2, 16, num_depositions) for num_depositions in range(22)
# ] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # ~~~~ n/a ~~~~
#     0,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     0,  # 8,  # 8,  # 0,
#     3,  # 2,  # 4,  # 2,
#     3,  # 4,  # 4,  # 0,
#     3,  # 9,  # 4,  # 1,
#     3,  # 12, # 4,  # 0,
#     3,  # 3,  # 4,  # 3,
#     3,  # 0,  # 4,  # 0,
#     3,  # 5,  # 4,  # 1,
#     3,  # 8,  # 4,  # 0,
#     3,  # 10, # 4,  # 2,
#     3,  # 4,  # 4,  # 0,
#     3,  # 13, # 4,  # 1,
#     3,  # 12, # 4,  # 0,
#     3,  # 4,  # 2,  # 4,
#     3,  # 0,  # 2,  # 0,
#     3,  # 1,  # 2,  # 1,
#     3,  # 8,  # 2,  # 0,
#     3,  # 6,  # 2,  # 2,
#     3,  # 0,  # 2,  # 0,
# ]
# assert [
#     get_deposition_rank_at_site(15, 16, num_depositions)
#     for num_depositions in range(22)
# ] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # ~~~~ n/a ~~~~
#     0,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     0,  # 8,  # 8,  # 0,
#     0,  # 2,  # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     0,  # 9,  # 4,  # 1,
#     0,  # 12, # 4,  # 0,
#     0,  # 3,  # 4,  # 3,
#     0,  # 0,  # 4,  # 0,
#     0,  # 5,  # 4,  # 1,
#     0,  # 8,  # 4,  # 0,
#     0,  # 10, # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     0,  # 13, # 4,  # 1,
#     0,  # 12, # 4,  # 0,
#     0,  # 4,  # 2,  # 4,
#     0,  # 0,  # 2,  # 0,
#     0,  # 1,  # 2,  # 1,
#     0,  # 8,  # 2,  # 0,
#     0,  # 6,  # 2,  # 2,
#     0,  # 0,  # 2,  # 0,
# ]
# assert [
#     get_deposition_rank_at_site(8, 16, num_depositions) for num_depositions in range(22)
# ] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # ~~~~ n/a ~~~~
#     0,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     2,  # 8,  # 8,  # 0,
#     2,  # 2,  # 4,  # 2,
#     2,  # 4,  # 4,  # 0,
#     2,  # 9,  # 4,  # 1,
#     2,  # 12, # 4,  # 0,
#     2,  # 3,  # 4,  # 3,
#     2,  # 0,  # 4,  # 0,
#     2,  # 5,  # 4,  # 1,
#     10, # 8,  # 4,  # 0,
#     10, # 10, # 4,  # 2,
#     10, # 4,  # 4,  # 0,
#     10, # 13, # 4,  # 1,
#     10, # 12, # 4,  # 0,
#     10, # 4,  # 2,  # 4,
#     10, # 0,  # 2,  # 0,
#     10, # 1,  # 2,  # 1,
#     18, # 8,  # 2,  # 0,
#     18, # 6,  # 2,  # 2,
#     18, # 0,  # 2,  # 0,
# ]
# assert [
#     get_deposition_rank_at_site(9, 16, num_depositions) for num_depositions in range(23)
# ] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # ~~~~ n/a ~~~~
#     0,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     0,  # 8,  # 8,  # 0,
#     0,  # 2,  # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     5,  # 9,  # 4,  # 1,
#     5,  # 12, # 4,  # 0,
#     5,  # 3,  # 4,  # 3,
#     5,  # 0,  # 4,  # 0,
#     5,  # 5,  # 4,  # 1,
#     5,  # 8,  # 4,  # 0,
#     5,  # 10, # 4,  # 2,
#     5,  # 4,  # 4,  # 0,
#     5,  # 13, # 4,  # 1,
#     5,  # 12, # 4,  # 0,
#     5,  # 4,  # 2,  # 4,
#     5,  # 0,  # 2,  # 0,
#     5,  # 1,  # 2,  # 1,
#     5,  # 8,  # 2,  # 0,
#     5,  # 6,  # 2,  # 2,
#     5,  # 0,  # 2,  # 0,
#     21, # 9,  # 2,  # 1,
# ]

pylib.jupyter_hide_toggle()


# 64-bit surface


In [ ]:
surface_size = 64
num_generations = 1000

surface_timestamps = [0] * surface_size
surface_values = [random.choice([0, 1]) for __ in range(surface_size)]

pylib.jupyter_hide_toggle()


In [ ]:
surface_timestamps_records = []
for generation in tqdm(range(num_generations)):
    target_site = get_deposition_site_at_rank(generation, surface_size)
    surface_values[target_site] = random.choice([0, 1])
    surface_timestamps[target_site] = generation

#     assert [
#         get_deposition_rank_at_site(site, surface_size, generation + 1)
#         for site in range(surface_size)
#     ] == surface_timestamps

    for site, timestamp in enumerate(surface_timestamps):
        surface_timestamps_records.append(
            {
                "generation": generation,
                "site": site,
                "timestamp": timestamp,
            }
        )

pylib.jupyter_hide_toggle()


In [ ]:
surface_timestamps_df = pd.DataFrame.from_records(surface_timestamps_records)
surface_timestamps_df

pylib.jupyter_hide_toggle()


In [ ]:
sns.scatterplot(
    data=surface_timestamps_df,
    x="timestamp",
    y="generation",
    hue="site",
).invert_yaxis()

pylib.jupyter_hide_toggle()


In [ ]:
sns.scatterplot(
    data=surface_timestamps_df,
    x="generation",
    y="timestamp",
    hue="site",
)

pylib.jupyter_hide_toggle()


In [ ]:
ax = sns.scatterplot(
    data=surface_timestamps_df,
    x="generation",
    y="site",
    hue="timestamp",
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

pylib.jupyter_hide_toggle()


# 16-bit surface


In [ ]:
surface_size = 16
num_generations = 135

surface_timestamps = [0] * surface_size
surface_values = [random.choice([0, 1]) for __ in range(surface_size)]

pylib.jupyter_hide_toggle()


In [ ]:
surface_timestamps_records = []
for generation in tqdm(range(num_generations)):
    target_site = get_deposition_site_at_rank(generation, surface_size)
    surface_values[target_site] = random.choice([0, 1])
    surface_timestamps[target_site] = generation

#     assert [
#         get_deposition_rank_at_site(site, surface_size, generation + 1)
#         for site in range(surface_size)
#     ] == surface_timestamps

    for site, timestamp in enumerate(surface_timestamps):
        surface_timestamps_records.append(
            {
                "generation": generation,
                "site": site,
                "timestamp": timestamp,
            }
        )

pylib.jupyter_hide_toggle()


In [ ]:
surface_timestamps_df = pd.DataFrame.from_records(surface_timestamps_records)
surface_timestamps_df

pylib.jupyter_hide_toggle()


In [ ]:
sns.scatterplot(
    data=surface_timestamps_df,
    x="timestamp",
    y="generation",
    hue="site",
).invert_yaxis()

pylib.jupyter_hide_toggle()


In [ ]:
sns.scatterplot(
    data=surface_timestamps_df,
    x="generation",
    y="timestamp",
    hue="site",
)

pylib.jupyter_hide_toggle()


In [ ]:
ax = sns.scatterplot(
    data=surface_timestamps_df,
    x="generation",
    y="site",
    hue="timestamp",
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

pylib.jupyter_hide_toggle()


# 8-bit surface


In [ ]:
surface_size = 8
num_generations = 50

surface_timestamps = [0] * surface_size
surface_values = [random.choice([0, 1]) for __ in range(surface_size)]

pylib.jupyter_hide_toggle()


In [ ]:
surface_timestamps_records = []
for generation in tqdm(range(num_generations)):
    target_site = get_deposition_site_at_rank(generation, surface_size)
    surface_values[target_site] = random.choice([0, 1])
    surface_timestamps[target_site] = generation

#     assert [
#         get_deposition_rank_at_site(site, surface_size, generation + 1)
#         for site in range(surface_size)
#     ] == surface_timestamps

    for site, timestamp in enumerate(surface_timestamps):
        surface_timestamps_records.append(
            {
                "generation": generation,
                "site": site,
                "timestamp": timestamp,
            }
        )

pylib.jupyter_hide_toggle()


In [ ]:
surface_timestamps_df = pd.DataFrame.from_records(surface_timestamps_records)
surface_timestamps_df

pylib.jupyter_hide_toggle()


In [ ]:
sns.scatterplot(
    data=surface_timestamps_df,
    x="timestamp",
    y="generation",
    hue="site",
).invert_yaxis()

pylib.jupyter_hide_toggle()


In [ ]:
sns.scatterplot(
    data=surface_timestamps_df,
    x="generation",
    y="timestamp",
    hue="site",
)

pylib.jupyter_hide_toggle()


In [ ]:
ax = sns.scatterplot(
    data=surface_timestamps_df,
    x="generation",
    y="site",
    hue="timestamp",
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

pylib.jupyter_hide_toggle()
